In [1]:
import numpy as np
import import_ipynb
import os
from lab1 import *

importing Jupyter notebook from lab1.ipynb


In [25]:
def mul_sequence(*args):
    result, flops, mults = None, 0, 0
    for arg in args:
        if result is None:
            result = arg
        else:
            result, f, m = strassen_multiply(result, arg)
            flops += f
            mults += m
    return result, flops, mults

In [7]:
def divide(A, X, Y):
    return A[:X//2,:Y//2],A[:X//2,Y//2:],A[X//2:,:Y//2],A[X//2:,Y//2:]

In [35]:
def inverse(A):
    if A is None:
        return 0, 0, 0
    X,Y = A.shape[0],A.shape[1]
    if X != Y:
        raise ValueError
    if X==0 or Y==0:
        raise ArithmeticError
    if X==1 and Y==1:
        return 1/A, 1, 1
    else:        
        A11,A12,A21,A22 = divide(A, X, Y)

        inv_A11, flops, mults = inverse(A11)
        
        S22_partly, f, m = mul_sequence(A21,inv_A11,A12)
        S22 = A22 - S22_partly
        flops += (A22.size+f)
        mults += m
        flops += f
        
        inv_S22, f, m = inverse(S22)
        mults += m
        flops += f
        
        B11_partly, B11f, B11m = mul_sequence(inv_A11, A12, inv_S22, A21, inv_A11)
        B11 = inv_A11 + B11_partly
        flops += B11.size
        
        B12, B12f, B12m = mul_sequence(-inv_A11, A12, inv_S22)
        B21, B21f, B21m = mul_sequence(-inv_S22, A21, inv_A11)
        B22 = inv_S22
        
        U = np.hstack((B11,B12))
        L = np.hstack((B21,B22))
        flops += sum((B12f, B21f, B11f))
        mults += sum((B12m, B21m, B11m))
        return np.vstack((U,L)), flops, mults

In [39]:
def LU(A):
    if A is None:
        return 0, 0, 0, 0
    X,Y = A.shape[0],A.shape[1]
    if X != Y:
        raise ValueError
    if X==0 or Y==0:
        raise ArithmeticError
    if X==1 and Y==1:
        return np.array([[1]]), A, 1, 1
    else:        
        A11,A12,A21,A22 = divide(A, X, Y)

        L11, U11, flops, mults = LU(A11)
        
        inv_U11, fu, mu = inverse(U11)
        inv_L11, fl, ml = inverse(L11)
        flops += (fu + fl)
        mults += (mu + ml)
        
        S_partly, Sf, Sm = mul_sequence(A21, inv_U11, inv_L11, A12)
        S = A22 - S_partly
        flops += (Sf + A22.size)
        mults += Sm
        
        Ls, Us, f, m= LU(S)
        flops += f
        mults += m
        
        U12, f, m = mul_sequence(inv_L11,A12)
        flops += f
        mults += m
        U21 = np.zeros((Us.shape[0],U11.shape[1]))
        
        L12 = np.zeros((L11.shape[0],Ls.shape[1]))
        L21, f, m = mul_sequence(A21, inv_U11)
        flops += f
        mults += m
        
        Uu = np.hstack((U11,U12))
        Ul = np.hstack((U21,Us))
        U = np.vstack((Uu,Ul))
        
        Lu = np.hstack((L11,L12))
        Ll = np.hstack((L21,Ls))
        L = np.vstack((Lu,Ll))
        
        return L, U, flops, mults

In [46]:
def det(A):
    if A is None:
        return 0, 0, 0
    X,Y = A.shape[0],A.shape[1]
    if X != Y:
        raise ValueError
    if X==0 or Y==0:
        raise ArithmeticError
    if X==1 and Y==1:
        return A[0,0], 0, 0
    else:        
        A11,A12,A21,A22 = divide(A, X, Y)
        L11, U11, flops, mults= LU(A11)
        
        inv_U11, fu, mu = inverse(U11)
        inv_L11, fl, ml = inverse(L11)
        
        flops += fu + fl
        mults += mu + ml
        
        S_partly, Sf, Sm = mul_sequence(A21, inv_U11, inv_L11, A12)
        S = A22 - S_partly
        flops += (Sf + A22.size)
        mults += Sm
        
        
        det_S, Sf, Sm = det(S)
        diagonals_product = np.prod(np.diagonal(U11))*np.prod(np.diagonal(L11))
        return diagonals_product*det_S, flops + Sf + np.diagonal(U11).size, mults + Sm + np.diagonal(U11).size

In [36]:
A = np.random.random((3,3))

In [37]:
inverse(A)

(array([[ -2.9041762 ,  31.7974454 ,  -1.90818211],
        [  0.94226952,   5.677443  ,  -2.46207263],
        [  6.23277898, -71.79050831,   8.21646924]]),
 62,
 40)

In [38]:
np.linalg.inv(A)

array([[ -2.9041762 ,  31.7974454 ,  -1.90818211],
       [  0.94226952,   5.677443  ,  -2.46207263],
       [  6.23277898, -71.79050831,   8.21646924]])

In [40]:
L, U, flops, mults = LU(A)

In [41]:
L, U

(array([[1.        , 0.        , 0.        ],
        [0.17745436, 1.        , 0.        ],
        [0.79191674, 8.73739148, 1.        ]]),
 array([[ 0.81479825,  0.7782775 ,  0.42243926],
        [ 0.        , -0.06315265, -0.01892375],
        [ 0.        ,  0.        ,  0.12170678]]))

In [42]:
L@U-A

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.11022302e-16, 0.00000000e+00, 0.00000000e+00]])

In [43]:
flops, mults

(29, 24)

In [47]:
det(A)

(-0.006262624938454663, 24, 19)

In [48]:
np.linalg.det(A)

-0.00626262493845467

In [20]:
A

array([[0.12222078, 0.33992219, 0.5261818 ],
       [0.08178844, 0.39596466, 0.98165937],
       [0.15982277, 0.34938962, 0.20765982]])